In [ ]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta, date
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np
from haversine import haversine_vector, Unit

In [ ]:
path_to_data='/scratch/spf248/covid/data'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
day=list(range(8,20))
night=list(range(8))+list(range(20,24))
inputs_folders=sorted([x.split('/')[-1][:-2] for x in glob(os.path.join(path_to_data,source,'s3',country,'*'))])
start_date=pd.to_datetime(inputs_folders[0],format='%Y%m%d').strftime('%m/%d/%Y')
end_date=pd.to_datetime(inputs_folders[-1],format='%Y%m%d').strftime('%m/%d/%Y')
print('Start:',start_date)
print('End:',end_date)
min_acc=10000

# Compute distance traveled

In [ ]:
print('Import')
start = timer()
data_dirs=glob(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
pings_geocoded = pd.concat(
    pd.read_parquet(parquet_file)
    for data_dir in data_dirs for parquet_file in Path(data_dir).glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Process')
start = timer()
pings_geocoded.drop(['classification_type',admin_id],1,inplace=True)
pings_geocoded.sort_values(by=['cuebiq_id','time'],inplace=True)
pings_geocoded['point'] = list(zip(pings_geocoded.latitude,pings_geocoded.longitude))
pings_geocoded.drop(['latitude','longitude'],1,inplace=True)
pings_geocoded=pd.concat([pings_geocoded,pings_geocoded.groupby('cuebiq_id')[['accuracy','point']].shift(-1).rename(columns=lambda x:x+'_destination')],1)
pings_geocoded=pings_geocoded[(np.abs(pings_geocoded.accuracy)<=min_acc)&(np.abs(pings_geocoded.accuracy_destination)<=min_acc)].copy()
pings_geocoded['hour']=pings_geocoded.time.dt.hour
pings_geocoded.dropna(inplace=True)
pings_geocoded.reset_index(drop=True,inplace=True)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Compute distance')
start = timer()
pings_geocoded['distance']=haversine_vector(pings_geocoded['point'].values.tolist(),pings_geocoded['point_destination'].values.tolist(),Unit.KILOMETERS)
pings_geocoded.drop(['point','point_destination'],1,inplace=True)
print("Done in", round(timer()-start), "sec") 

In [ ]:
print('Aggregate Distances')
start = timer()
distances_date_id=pings_geocoded.groupby([pd.Grouper(key='time',freq='D'),'cuebiq_id'])['distance'].agg(['mean','median','max','count']).reset_index().rename(columns={'time':'date'})
distances_daytime_id=pings_geocoded[pings_geocoded['hour'].isin(day)].groupby([pd.Grouper(key='time',freq='D'),'cuebiq_id'])['distance'].agg(['mean','median','max','count']).reset_index().rename(columns={'time':'date'})
distances_nighttime_id=pings_geocoded[pings_geocoded['hour'].isin(night)].groupby([pd.Grouper(key='time',freq='D'),'cuebiq_id'])['distance'].agg(['mean','median','max','count']).reset_index().rename(columns={'time':'date'})
print("Done in", round(timer()-start), "sec") 

In [ ]:
print('Process')
start = timer()
distances_date_id=distances_date_id[(distances_date_id['date']>=pd.to_datetime(start_date))&(distances_date_id['date']<=pd.to_datetime(end_date))].reset_index(drop=True).copy()
distances_daytime_id=distances_daytime_id[(distances_daytime_id['date']>=pd.to_datetime(start_date))&(distances_daytime_id['date']<=pd.to_datetime(end_date))].reset_index(drop=True).copy()
distances_nighttime_id=distances_nighttime_id[(distances_nighttime_id['date']>=pd.to_datetime(start_date))&(distances_nighttime_id['date']<=pd.to_datetime(end_date))].reset_index(drop=True).copy()
distances_date_id.rename(columns={'mean':'mean_distance','median':'median_distance','max':'max_distance','count':'n_pings'},inplace=True)
distances_daytime_id.rename(columns={'mean':'mean_distance','median':'median_distance','max':'max_distance','count':'n_pings'},inplace=True)
distances_nighttime_id.rename(columns={'mean':'mean_distance','median':'median_distance','max':'max_distance','count':'n_pings'},inplace=True)
print("Done in", round(timer()-start), "sec") 

In [ ]:
print('Save')
start = timer()
distances_date_id.to_csv(os.path.join(path_to_data,source,'processed',country,'distances_date_id.csv'))
distances_daytime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'distances_daytime_id.csv'))
distances_nighttime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'distances_nighttime_id.csv'))
print("Done in", round(timer()-start), "sec")

# Figures